In [32]:
#本次实验环境为python 3.7,使用pymysql包
import pymysql as mdb
import re
import datetime
import time
import _pickle as cPickle
import scipy.stats
import math
import time
import numpy

In [33]:
config={
            'host':'127.0.0.1',
            'port':3306,
            'user':'root',
            'passwd':'20000822lty666',
            'db':'eams',
            'charset':'utf8'
            }

conn=mdb.connect(**config) #建立连接
conn.autocommit(True)
cursor=conn.cursor() 
class school:
    __school_dic={}
    def __init__(self):
        self.school_name=input("请输入学校名称：")
        if self.school_name not in self.__school_dic:
            self.school_id=len(self.__school_dic)+1
            self.__school_dic[self.school_name]=self.school_id
            sql='INSERT INTO `eams`.`school`(`school_id`, `school_name`) values(%d,\'%s\')'%(self.school_id,self.school_name)
            cursor.execute(sql)
        else:
            self.school_id=self.__school_dic[self.school_name]
class academy(school):
    __academy_dic={}
    def __init__(self):
        school.__init__(self)
        #school.school_id-=1
        self.academy_name=input("请输入学院名称：")
        if self.academy_name not in self.__academy_dic:
            self.academy_id=len(self.__academy_dic)+1
            self.__academy_dic[self.academy_name]=self.academy_id
            sql='INSERT INTO `eams`.`academy`(`academy_id`, `academy_name`,`school_id`) values(%d,\'%s\',%d)'%(self.academy_id,self.academy_name,self.school_id)
            cursor.execute(sql)
        else:
            self.academy_id=self.__academy_dic[self.academy_name]
class major(academy):
    __major_dic={}
    def __init__(self):
        academy.__init__(self)
        #academy.school_id-=1
        self.major_name=input("请输入专业名称：")
        if self.major_name not in self.__major_dic:
            self.major_id=len(self.__major_dic)+1
            self.__major_dic[self.major_name]=self.major_id
            sql='INSERT INTO `eams`.`major`(`major_id`, `major_name`,`school_id`,`academy_id`) values(%d,\'%s\',%d,%d)'%(self.major_id,self.major_name,self.school_id,self.academy_id)
            cursor.execute(sql)
        else:
            self.major_id=self.__major_dic[self.major_name]
        



In [3]:
def clearDataBase():
    #清空数据库的函数
    sql="delete from stuattend"
    cursor.execute(sql)
    sql="delete from attend"
    cursor.execute(sql)
    sql="delete from answer"
    cursor.execute(sql)
    sql="delete from problem"
    cursor.execute(sql)
    sql="delete from exam"
    cursor.execute(sql)
    sql="delete from courseselect"
    cursor.execute(sql)
    sql="delete from courseteach"
    cursor.execute(sql)
    sql="delete from coursepage"
    cursor.execute(sql)
    sql="delete from course"
    cursor.execute(sql)
    sql="delete from teacher"
    cursor.execute(sql)
    sql="delete from student"
    cursor.execute(sql)
    sql="delete from admin"
    cursor.execute(sql)
    sql="delete from major"
    cursor.execute(sql)
    sql="delete from academy"
    cursor.execute(sql)
    sql="delete from school"
    cursor.execute(sql)
    
class student:
    def __init__(self,stu_id,stu_name,stu_pwd,school_id,academy_id,major_id):
        self.stu_id=stu_id
        self.stu_name=stu_name
        self.stu_pwd=stu_pwd
        self.school_id=school_id
        self.academy_id=academy_id
        self.major_id=major_id
        
    def selectCourse(self):
        sql="select * from course where school_id=%d and academy_id=%d and major_id=%d"%(self.school_id,self.academy_id,self.major_id)
        cursor.execute(sql)
        print('所有可选课程为：')
        
        #显示所有可选课程信息
        courseSel=cursor.fetchall()
        for data in courseSel:
            if data[-1]:
                print("课程代码：",data[0])
                print("课程名称：",data[1])
                print("开课时间：星期",data[2])
                print("授课时间：",data[3])
                sql="select major_name from major where major_id=%d and school_id=%d and academy_id=%d"%(data[-2],self.school_id,self.academy_id)
                cursor.execute(sql)
                print("开课专业：",cursor.fetchall()[0][0])
                sql="select teacher_id from courseteach where course_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(data[0],self.school_id,self.academy_id,self.major_id)
                cursor.execute(sql)
                sql="select teacher_name from teacher where teacher_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(cursor.fetchall()[0][0],self.school_id,self.academy_id,self.major_id)
                cursor.execute(sql)
                print("任课教师为：",cursor.fetchall()[0][0])
                print()
        
        while True:
            select=input("请输入想选择课程的课程代码： q)退出")
            if select =='q' or select=='Q':
                break
            try:
                select=int(select)
            except:
                break
            if select not in map(lambda x:x[0],courseSel):
                print("输入课程代码错误！")
            else:
                #查找想选课程的授课时间
                sql="select course_day,course_time from course where course_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(select,self.school_id,self.academy_id,self.major_id)
                cursor.execute(sql)
                day,time=cursor.fetchall()[0]
                
                
                #先查找所有已选课程
                sql="select course_id from courseselect where stu_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(self.stu_id,self.school_id,self.academy_id,self.major_id)
                cursor.execute(sql)
                #courses存储所有已选课程id
                courses=[]
                for course in map(lambda x:x[0],cursor.fetchall()):
                    sql="select course_day from course where course_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(course,self.school_id,self.academy_id,self.major_id)
                    cursor.execute(sql)
                    if cursor.fetchall()[0][0]==day:
                        courses.append(course)
                
                t=1
                for course in courses:
                    
                    
                    
                    sql="select course_time from course where course_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(course,self.school_id,self.academy_id,self.major_id)
                    cursor.execute(sql)
                    cou_time=cursor.fetchall()[0][0]
                    #判断是否与已有课程时间段重合 默认一节课90分钟
                    
                    #计算差值：minute
                    minute=abs(cou_time-time)
                    
                    if minute<datetime.timedelta(seconds=5400):
                        print("与已选课程冲突！请重新选择：")
                        t=0
                        break
                    
                        
                if t:    
                    sql="insert into courseselect(stu_id,course_id,school_id,academy_id,major_id) values (%d,%d,%d,%d,%d)"%(self.stu_id,select,self.school_id,self.academy_id,self.major_id)
                    try:
                        cursor.execute(sql)
                        print("选课成功！")
                        break
                    except:
                        print("未知错误！")
                        break
    
    
    def downloadFile(self):
        #学生下载课件
        
        #先查找指定课程
        sql="select course_id from course where school_id=%d and academy_id=%d and major_id=%d "%(self.school_id,self.academy_id,self.major_id)
        cursor.execute(sql)
        courses=list(map(lambda x:x[0],cursor.fetchall()))
        while True:
            try:
                course_id=int(input("请输入课件所属课程代码： int类型)"))
                if course_id in courses:
                    break
                else:
                    print("课程代码不存在！")
            except:
                print("课程代码无效，请重新输入！")
        
        sql='select course_id,file_name from coursepage where course_id =%d and school_id=%d and academy_id=%d and major_id=%d'%(course_id,self.school_id,self.academy_id,self.major_id) 
        cursor.execute(sql)
        print("可下载课件为：")
        for course_id,file_name in cursor.fetchall():
            print("课程代码：",course_id,"   文件名称："+file_name)
        
        sql="select file_name,file from coursepage where course_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(course_id,self.school_id,self.academy_id,self.major_id)
        cursor.execute(sql)
        
        content={}
        for file_name,file in cursor.fetchall():
            content[file_name]=file
        
        while True:
            name=input("请输入要下载的课件名称:  q)退出")
            if name=='q' or name=='Q':
                break
            if name not in content:
                print("课件不存在！")
            else:
                address=input("请输入下载路径：")
                with open(address+name,'w') as f:
                    f.write(cPickle.loads(file))
                    print("下载成功！")
                    break
    
    def doTest(self):
        #先查找所有已选课程
        
        sql="select course_id from courseselect where stu_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(self.stu_id,self.school_id,self.academy_id,self.major_id)
        cursor.execute(sql)
        courses=list(map(lambda x:x[0],list(cursor.fetchall())))
        
        #查找所有有考试的课程
        sql="select * from exam"
        cursor.execute(sql)
        examdata=list(cursor.fetchall())
        for exam in examdata:
            if exam[-4] in courses and exam[-3]==self.school_id and exam[-2]==self.academy_id and exam[-1]==self.major_id:
                
                print("考试代码：",exam[0])
                print("课程代码：",exam[-4])
                print("考试起始时间：",exam[1])
                print("考试结束时间：",exam[2])
                
                if datetime.datetime.now()<exam[1]:
                    print("考试未开始！")
                elif datetime.datetime.now()<exam[2]:
                    print("考试进行中，可以进行考试！")
                else:
                    print("考试已结束！")
        while True:
            exam_id=input("如需进行考试，请输入考试代码；否则输入 q)退出")
            if exam_id=='q' or exam_id=='Q':
                break
            else:
                exam_id=int(exam_id)
                for exam in examdata:
                    if exam[0]==exam_id:
                        #输入考试代码有效，先查看考试状态
                        if datetime.datetime.now()<exam[1] or datetime.datetime.now()>exam[2]:
                            print("未在考试时间，无法考试！")
                        else:
                            #进行考试
                            sql="select problem_id,content from problem where exam_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(exam_id,self.school_id,self.academy_id,self.major_id)
                            cursor.execute(sql)
                            problems=cursor.fetchall()
                            for problem in problems:
                                print("题目%d:"%problem[0],problem[1])
                                ans=input("我的答案：")
                                sql="insert into answer(stu_id,problem_id,answer,grade,exam_id,state,school_id,academy_id,major_id) values(%d,%d,'%s',0,%d,%d,%d,%d,%d)"%(self.stu_id,problem[0],ans,exam_id,0,self.school_id,self.academy_id,self.major_id)
                                cursor.execute(sql)
                            print("考试完成！")
                            return 
                print("未查询到该考试，请重新输入！")
                    
    def signAttend(self):
        #学生签到
        
        #先找出已选课程中，所有可签到的签到
        sql="select course_id from courseselect where stu_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(self.stu_id,self.school_id,self.academy_id,self.major_id)
        cursor.execute(sql)
        
        courses=list(map(lambda x:x[0],list(cursor.fetchall())))
        
        print("所有签到为：")
        
        data=[]
        for course in courses:
            sql="select * from attend where course_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(course,self.school_id,self.academy_id,self.major_id)
            cursor.execute(sql)
            att=cursor.fetchall()
            for at in att:
                if at[4]<=datetime.datetime.now()<=at[5]:
                    print("签到代码：",at[0])
                    print("签到名称：",at[1])
                    print("签到开始时间：",at[4])
                    print("签到结束时间：",at[5])
                    data.append(at)
        
        while True:
            attend_id=input("请输入想签到的签到代码：   q)退出")
            if attend_id=='q' or attend_id=='Q':
                break
            else:
                attend_id=int(attend_id)
            
            #先查看该生是否已经签过了这次签到
            sql="select * from stuattend where stu_id=%d and attend_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(self.stu_id,attend_id,self.school_id,self.academy_id,self.major_id)
            cursor.execute(sql)
            if len(cursor.fetchall()):
                #已经签过到
                print("已经签过到了哦！")
            else:
                #进行签到
                sql="insert into stuattend(stu_id,attend_id,school_id,academy_id,major_id) values(%d,%d,%d,%d,%d)"%(self.stu_id,attend_id,self.school_id,self.academy_id,self.major_id)
                cursor.execute(sql)
                print("签到成功！")
    
    def showSchedule(self):
        #查询所有已选课程的课程代码
        print("您的课程表如下：")
        sql="select course_id from courseselect where stu_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(self.stu_id,self.school_id,self.academy_id,self.major_id)
        cursor.execute(sql)
        courses=list(map(lambda x:x[0],cursor.fetchall()))
        data=[]
        
        #找到每个课程的所有信息
        for course in courses:
            sql="select * from course where course_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(course,self.school_id,self.academy_id,self.major_id)
            cursor.execute(sql)
            data.append(cursor.fetchall()[0])
            
        #将课程按照时间顺序排序
        data.sort(key=lambda x:(x[2],x[3]))
        
        #逐一显示
        for course in data:
            print("星期%d %s %s"%(course[2],course[3],course[1]))
        
        
class teacher:
    def __init__(self,tea_id,tea_name,tea_pwd,school_id,academy_id,major_id):
        self.tea_id=tea_id
        self.tea_name=tea_name
        self.tea_pwd=tea_pwd
        self.school_id=school_id
        self.academy_id=academy_id
        self.major_id=major_id
    
    def addCourse(self):
        while True:
            try:
                id=int(input("请输入新开课程代码： int类型)"))
                break
            except:
                print("课程代码无效，请重新输入！")
        name=input("请输入课程名称：")
        day=int(input("请输入开课日期 （1~7 ：星期一~日）"))
        hour,minute,second=input("请输入具体开课时间 如15 00 00表示15:00:00开始授课").split()
        
        sql="insert into course(course_id,course_name,course_day,course_time,school_id,academy_id,major_id,verify) values(%d,\'%s\',%d,\'%s:%s:%s\',%d,%d,%d,0)"%(id,name,day,hour,minute,second,self.school_id,self.academy_id,self.major_id)
        try:
            cursor.execute(sql)
            print("开课成功！(待审核)")
            sql="insert into courseteach(teacher_id,course_id,school_id,academy_id,major_id) values(%d,%d,%d,%d,%d)"%(self.tea_id,id,self.school_id,self.academy_id,self.major_id)
            try:
                cursor.execute(sql)
                print("录入系统成功！")
            except:
                print("录入系统失败，请尝试手动录入！")
        except:
            print("未知错误！")
            
    def uploadFile(self):
        while True:
            try:
                id=int(input("请输入课件所属课程代码： int类型)"))
                break
            except:
                print("课程代码无效，请重新输入！")
        while True:
            try:
                file_id=int(input("请输入课件代码： int类型)"))
                break
            except:
                print("课件代码无效，请重新输入！")
        while True:        
            file_name=input("请输入上传课件文件名（带拓展名）： q)退出")
            if file_name=='q' or file_name=='Q':
                break
            file_address=input("请输入上传课件路径：")
            
            file=open(file_address,'r')
            print("读取成功！")
            file_content=file.read()
                
            file.close()
            bina=cPickle.dumps(file_content,2)                   
                
            sql="insert into coursepage(course_id,file_name,file,school_id,academy_id,major_id,file_id) values(%s,%s,%s,%s,%s,%s,%s)"
            va=(id,file_name,bina,self.school_id,self.academy_id,self.major_id,file_id)
            try:
                cursor.execute(sql,va)
                print("上传成功！")
            except:
                print("上传失败")
         
    def launchTest(self):
        #教师发起考试
        while True:
            try:
                course_id=int(input("请输入要考试的课程代码： int类型)"))
                break
            except:
                print("课程代码无效，请重新输入！")
        while True:
            try:
                exam_id=int(input("请输入考试代码： int类型)"))
                break
            except:
                print("考试代码无效，请重新输入！")
        year_start,month_start,day_start,hour_start,minute_start,second_start=map(int,input("请输入具体考试开始时间 如2020 6 28 15 00 00表示2020年6月28日15:00:00开始考试").split())
        year_end,month_end,day_end,hour_end,minute_end,second_end=map(int,input("请输入具体考试结束时间 如2020 6 28 16 00 00表示2020年6月28日16:00:00结束考试").split())
        
        time_start=datetime.datetime(year_start,month_start,day_start,hour_start,minute_start,second_start)
        time_end=datetime.datetime(year_end,month_end,day_end,hour_end,minute_end,second_end)
        
        
        
        sql="insert into exam(exam_id,exam_starttime,exam_endtime,teacher_id,course_id,school_id,academy_id,major_id) values(%s,%s,%s,%s,%s,%s,%s,%s)"
        va=(exam_id,time_start,time_end,self.tea_id,course_id,self.school_id,self.academy_id,self.major_id)
        
        cursor.execute(sql,va)
        print("发起考试成功！进入出题界面：")
        while True:
            problem_id=input("请输入题号： q)退出")
            if problem_id=='q' or problem_id=='Q':
                break
            else:
                problem_id=int(problem_id)
            content=input("请输入题目：")
            
            sql="insert into problem(problem_id,content,exam_id,school_id,academy_id,major_id) values (%d,'%s',%d,%d,%d,%d)"%(problem_id,content,exam_id,self.school_id,self.academy_id,self.major_id)
            cursor.execute(sql)
            
        
        
    def evaluateTest(self):
        #批改试卷
        
        #先查找所有需要批改的考试
        
        sql="select exam_id,exam_starttime,exam_endtime,course_id from exam where teacher_id=%d and school_id=%d and academy_id=%d and major_id=%d "%(self.tea_id,self.school_id,self.academy_id,self.major_id)
        cursor.execute(sql)
        
        outcomes=cursor.fetchall()
        exams=[]
        print("所有需要批改的试卷：")
        for outcome in outcomes:
            if outcome[2]<datetime.datetime.now():
                exams.append(outcome[0])
                print("考试编号%d"%outcome[0])
        for exam in exams:
            
            print("批改考试编号%d的试卷："%exam)
            sql="select * from answer where exam_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(exam,self.school_id,self.academy_id,self.major_id)
            cursor.execute(sql)
            ans=list(cursor.fetchall())
            for an in ans:
                if an[-4]==0:
                    print("学生学号：",an[0]," 题目%d"%an[1],"答案：",an[2])
                    grade=float(input("得分："))
                    sql="update answer set grade=%d,state=1 where stu_id=%d and problem_id=%d and exam_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(grade,an[0],an[1],an[4],self.school_id,self.academy_id,self.major_id)
                    cursor.execute(sql)
                
            print("考试编号为%d的试卷已批改！"%exam)
            
        
    def launchAttend(self):
        
        #教师发起签到
        while True:
            try:
                course_id=int(input("请输入要发起签到的课程代码： int类型)"))
                break
            except:
                print("课程代码无效，请重新输入！")
        
        while True:
            try:
                attend_id=int(input("请输入签到代码： int类型)"))
                break
            except:
                print("签到代码无效，请重新输入！")
        
        attendname=input("请输入签到名称")
        year_start,month_start,day_start,hour_start,minute_start,second_start=map(int,input("请输入具体签到开始时间 如2020 6 28 15 00 00表示2020年6月28日15:00:00开始签到").split())
        year_end,month_end,day_end,hour_end,minute_end,second_end=map(int,input("请输入具体签到结束时间 如2020 6 28 16 00 00表示2020年6月28日16:00:00结束签到").split())
        
        time_start=datetime.datetime(year_start,month_start,day_start,hour_start,minute_start,second_start)
        time_end=datetime.datetime(year_end,month_end,day_end,hour_end,minute_end,second_end)
        
        sql="insert into attend(attend_id,attendname,course_id,teacher_id,starttime,endtime,school_id,academy_id,major_id) values(%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        va=(attend_id,attendname,course_id,self.tea_id,time_start,time_end,self.school_id,self.academy_id,self.major_id)
        cursor.execute(sql,va)
        
        print("发起签到成功！")
        
        
    def showSchedule(self):
        #查询所有已选课程的课程代码
        print("您的课程表如下：")
        sql="select course_id from courseteach where teacher_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(self.tea_id,self.school_id,self.academy_id,self.major_id)
        cursor.execute(sql)
        courses=list(map(lambda x:x[0],cursor.fetchall()))
        data=[]
        
        #找到每个课程的所有信息
        for course in courses:
            sql="select * from course where course_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(course,self.school_id,self.academy_id,self.major_id)
            cursor.execute(sql)
            data.append(cursor.fetchall()[0])
            
        #将课程按照时间顺序排序
        data.sort(key=lambda x:(x[2],x[3]))
        
        #逐一显示
        for course in data:
            print("星期%d %s %s"%(course[2],course[3],course[1]))
    
class admin:
    def __init__(self,admin_id,admin_name,admin_pwd,school_id,academy_id,major_id):
        self.admin_id=admin_id
        self.admin_name=admin_name
        self.admin_pwd=admin_pwd
        self.school_id=school_id
        self.academy_id=academy_id
        self.major_id=major_id

    def showSchedule(self):
        #管理员视角显示课表
        sql="select * from course where verify=1 and school_id=%d and academy_id=%d and major_id=%d"%(self.school_id,self.academy_id,self.major_id)
        cursor.execute(sql)
        data=list(cursor.fetchall())
        
        #将课程按照时间顺序排序
        data.sort(key=lambda x:(x[2],x[3]))
        
        for course in data:
            print("星期%d %s %s  课程代码：%d"%(course[2],course[3],course[1],course[0]))
            
            
    def supervise(self):
        #教学督察，查看课程互动情况
        print("所有正在进行的课程为：")
        self.showSchedule()
        
        while True:
            try:
                course_id=int(input("请输入要查看课程互动情况的课程代码： int类型)"))
                break
            except:
                print("课程代码无效，请重新输入！")
        
        #先查看该课程的所有签到情况
        sql="select * from attend where course_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(course_id,self.school_id,self.academy_id,self.major_id)
        cursor.execute(sql)
        att=cursor.fetchall()
        print("课程签到情况：")
        for at in att:
            print("签到代码：%d 签到名称：%s 签到起始时间：%s 签到结束时间：%s"%(at[0],at[1],at[4],at[5]))
        
        #查看该课程的所有测验情况
        sql="select * from exam where course_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(course_id,self.school_id,self.academy_id,self.major_id)
        cursor.execute(sql)
        exams=cursor.fetchall()
        print("课程测验情况：")
        for exam in exams:
            print("考试代码：%d  考试起始时间：%s 考试结束时间：%s"%(exam[0],exam[1],exam[2]))
        
        
        
        
    def staGrade(self):
        #系统统计不同课程的平均成绩
        
        course_id=int(input("请输入要查询的课程代码：  int类型)"))
        #找到该课程所有进行过的考试
        sql="select exam_id from exam where course_id=%s and school_id=%s and academy_id=%s and major_id=%s"
        va=(course_id,self.school_id,self.academy_id,self.major_id)
        cursor.execute(sql,va)
        
        exams=list(map(lambda x:x[0],list(cursor.fetchall())))
        
        exam_id=int(input("请输入要查询该课程的哪一次考试的考试代码：  int类型)"))
        if exam_id not in exams:
            print("未查询到本次考试！")
        else:
            #进行统计，分别统计各项数据
            #先求每个学生的总成绩
            scores={}
            
            sql="select stu_id,problem_id,grade from answer where exam_id=%d and state=1 and school_id=%d and academy_id=%d and major_id=%d"%(exam_id,self.school_id,self.academy_id,self.major_id)
            cursor.execute(sql)
            for stu_id,problem_id,grade in cursor.fetchall():
                if stu_id not in scores:
                    scores[stu_id]=grade
                else:
                    scores[stu_id]+=grade
            avg_score=numpy.mean(list(scores.values()))
            var_score=numpy.var(list(scores.values()))
            max_score=numpy.max(list(scores.values()))
            print("本次考试的平均分为：",avg_score)
            print("本次考试的方差为：",var_score)
            print("本次考试的最高分为：",max_score)
            
            #如果样本量足够大，可以检验是否
            if len(scores)>8:
                print("样本量大于8，可以进行正态分布检验：")
                p_value=scipy.stats.normaltest(list(scores.values())).pvalue
                print("p值为：",p_value)
                if p_value>0.05:
                    print("p值大于0.05，满足正态分布！")
                else:
                    print("p值小于等于0.05，不满足正态分布！")
    
    def staAttend(self):
        #统计指定课程的指定签到的出勤率
        course_id=int(input("请输入要查询的课程代码：  int类型)"))
        attend_id=int(input("请输入要查询的签到代码：  int类型)"))
        
        sql="select count(stu_id)/count(distinct(stu_id))-1 from(select courseselect.stu_id from courseselect where course_id=%d and school_id=%d and academy_id=%d and major_id=%d union all select stuattend.stu_id from stuattend where attend_id=%d and school_id=%d and academy_id=%d and major_id=%d) as a"%(course_id,self.school_id,self.academy_id,self.major_id,attend_id,self.school_id,self.academy_id,self.major_id)
        cursor.execute(sql)
        rate=cursor.fetchall()[0][0]
        print("本次签到出勤率为：",rate)
    
    def adjustCourse(self):
        
        #查询本专业所有课程信息
        sql="select * from course where school_id=%d and academy_id=%d and major_id=%d"%(self.school_id,self.academy_id,self.major_id)
        cursor.execute(sql)
        courses=list(cursor.fetchall())
        
        for course in courses:
            print("课程代码：",course[0])
            print("课程名称：",course[1])
            print("授课日期：",course[2])
            print("授课时间：",course[3])
            print("审核状态 0)未通过 1)通过",course[-1])
        while True:
            modify=input("请输入想要修改的内容： 0)课程名称 1)授课日期 2)授课时间 3)审核状态  q)退出")
        
            if modify=='0':
                while True:
                    try:
                        id=int(input("请输入要修改的课程代码："))
                        if id not in map(lambda x:x[0],courses):
                            print("输入课程代码无效，请重新输入！")
                            continue
                        else:
                            break
                    except:
                        print("请输入有效课程代码 int)")

                name=input("请输入新的课程名称：")
                sql="update course set course_name=\'%s\' where course_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(name,id,self.school_id,self.academy_id,self.major_id)
                try:
                    cursor.execute(sql)
                    print("修改成功！")
                except:
                    print("发生错误，修改课程名称未成功")
            elif modify=='1':
                while True:
                    try:
                        id=int(input("请输入要修改的课程代码："))
                        if id not in map(lambda x:x[0],courses):
                            print("输入课程代码无效，请重新输入！")
                            continue
                        else:
                            break
                    except:
                        print("请输入有效课程代码 int)")
                while True:
                    try:
                        day=int(input("请输入新的课程日期： 1~7)"))
                        if day>=1 and day<=7:
                            break
                        else:
                            print("超出范围，请重新输入")
                    except:
                        print("输入课程日期无效，请重新输入！")


                sql="update course set course_day=%d where course_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(day,id,self.school_id,self.academy_id,self.major_id)
                try:
                    cursor.execute(sql)
                    print("修改成功！")
                except:
                    print("发生错误，修改课程日期未成功")
            elif modify=='2':
                while True:
                    try:
                        id=int(input("请输入要修改的课程代码："))
                        if id not in map(lambda x:x[0],courses):
                            print("输入课程代码无效，请重新输入！")
                            continue
                        else:
                            break
                    except:
                        print("请输入有效课程代码 int)")
                while True:
                    try:
                        hour,minute,second=map(int,input("请输入新的授课时间: 例如 15 00 00即修改为15:00:00)").split())
                        if hour>24 or hour<0 or minute>60 or minute<0 or second>60 or second<0:
                            print("输入授课时间超出范围,请重新输入")
                        else:
                            break

                    except:
                        print("输入授课时间无效，请重新输入！")


                sql="update course set course_time=%s where course_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(datetime.time(hour,minute,second),id,self.school_id,self.academy_id,self.major_id)
                try:
                    cursor.execute(sql)
                    print("修改成功！")
                except:
                    print("发生错误，修改授课时间未成功")

            elif modify=='3':
                while True:
                    try:
                        id=int(input("请输入要修改的课程代码："))
                        if id not in map(lambda x:x[0],courses):
                            print("输入课程代码无效，请重新输入！")
                            continue
                        else:
                            break
                    except:
                        print("请输入有效课程代码 int)")
                while True:
                    try:
                        verify=int(input("请输入修改后的审核状态"))
                        if verify==1 or verify==0:
                            break
                        else:
                            print("非法状态，请重新输入！  0)审核未通过 1)审核通过")

                    except:
                        print("输入审核状态无效，请重新输入！")


                sql="update course set verify=%d where course_id=%d and school_id=%d and academy_id=%d and major_id=%d"%(verify,id,self.school_id,self.academy_id,self.major_id)
                try:
                    cursor.execute(sql)
                    print("修改成功！")
                except:
                    print("发生错误，修改审核状态未成功")
                    
            elif modify=='q' or modify=='Q':
                break
            else:
                print("无效输入！")
                
                


In [36]:
class eams:
    def __init__(self):
        config={
            'host':'127.0.0.1',
            'port':3306,
            'user':'root',
            'passwd':'20000822lty666',
            'db':'eams',
            'charset':'utf8'
            }

        self.conn=mdb.connect(**config) #建立连接
        self.conn.autocommit(True)
        cursor=self.conn.cursor() 
        self.cursor=cursor #获取游标
        
        self.major=major()
        self.major.stu=[]
        self.major.tea=[]
        self.major.admin=[]
        
    
    def compareIndex(self):
        #独立索引机制，验证索引的查询速率
        sql="select exam_starttime,exam_endtime from exam where exam_endtime<%s and exam_endtime>%s"
        va=(datetime.datetime(2020,6,28,0,0,0),datetime.datetime(2020,6,30,0,0,0))
        start_time=time.time()
        cursor.execute(sql,va)
       
        end_time=time.time()
        run_time=end_time-start_time
        print("不使用索引的运行时间为：",run_time)
        sql="alter table exam add index `index_exam`(exam_endtime,exam_starttime)"
        cursor.execute(sql)
        sql="select exam_starttime,exam_endtime from exam where exam_endtime<%s and exam_endtime>%s"
        start_time=time.time()
        cursor.execute(sql,va)
        end_time=time.time()
        run_time=end_time-start_time
        print("使用索引的运行时间为：",run_time)
    
    
    def run(self):
        #教务系统开始工作！
        
        print("欢迎使用上流教务管理系统！")
        while True:
            func=input("请输入想使用的功能 1)注册 2)登录 q)退出")
            if func=='q' or func=='Q':
                break
            elif func =='1':
                #注册功能
                self.register()
            
            elif func=='2':
                #登录
                while True:
                    chara=input("请选择登录角色： 1)学生 2)教师 3)管理员 q)退出")
                    if chara=='q' or chara=='Q':
                        break
                    elif chara=='1':

                        stu_index=0
                        flag=0
                        while True:
                            user_id=input("请输入您的学号:  q)退出")
                            if user_id=='q' or user_id=='Q':
                                break
                            user_id=int(user_id)
                            user_pwd=input("请输入密码：")
                            for i in range(len(self.major.stu)):
                                if self.major.stu[i].stu_id==user_id and self.major.stu[i].stu_pwd==user_pwd:
                                    stu_index=i
                                    flag=1
                                    break
                            if flag!=0:
                                print("登录成功！")
                                break
                            else:
                                print("用户名或密码错误，请重新收入！")

                        if flag==0:
                            break

                        while True:
                            fun=input("请输入您想使用的功能： 1)选课 2)上课（下载课件进行学习） 3)考试 4)签到 5)显示课程表 q)退出")
                            if fun=='1':
                                self.major.stu[stu_index].selectCourse()
                            elif fun=='2':
                                self.major.stu[stu_index].downloadFile()
                            elif fun=='3':
                                self.major.stu[stu_index].doTest()
                            elif fun=='4':
                                self.major.stu[stu_index].signAttend()
                            elif fun=='5':
                                self.major.stu[stu_index].showSchedule()
                            elif fun=='q' or fun=='Q':
                                break
                            else:
                                print("非法输入，请重新输入！")
                        
                    elif chara=='2':

                        tea_index=0
                        flag=0
                        while True:
                            user_id=input("请输入您的教工号:  q)退出")
                            if user_id=='q' or user_id=='Q':
                                break
                            user_id=int(user_id)
                            user_pwd=input("请输入密码：")
                            for i in range(len(self.major.tea)):
                                if self.major.tea[i].tea_id==user_id and self.major.tea[i].tea_pwd==user_pwd:
                                    tea_index=i
                                    flag=1
                                    break
                            if flag!=0:
                                print("登录成功！")
                                break
                            else:
                                print("用户名或密码错误，请重新收入！")

                        if flag==0:
                            break

                        while True:
                            fun=input("请输入您想使用的功能： 1)新开课程 2)上课（上传课件） 3)发起考试 4)批改试卷 5)发起签到 6)显示课程表 q)退出")
                            if fun=='1':
                                self.major.tea[tea_index].addCourse()
                            elif fun=='2':
                                self.major.tea[tea_index].uploadFile()
                            elif fun=='3':
                                self.major.tea[tea_index].launchTest()
                            elif fun=='4':
                                self.major.tea[tea_index].evaluateTest()
                            elif fun=='5':
                                self.major.tea[tea_index].launchAttend()
                            elif fun=='6':
                                self.major.tea[tea_index].showSchedule()
                            elif fun=='q' or fun=='Q':
                                break
                            else:
                                print("非法输入，请重新输入！")
                                
                    elif chara=='3':

                        admin_index=0
                        flag=0
                        while True:
                            user_id=input("请输入您的教工号:  q)退出")
                            if user_id=='q' or user_id=='Q':
                                break
                            user_id=int(user_id)
                            user_pwd=input("请输入密码：")
                            for i in range(len(self.major.admin)):
                                if self.major.admin[i].admin_id==user_id and self.major.admin[i].admin_pwd==user_pwd:
                                    admin_index=i
                                    flag=1
                                    break
                            if flag!=0:
                                print("登录成功！")
                                break
                            else:
                                print("用户名或密码错误，请重新收入！")

                        if flag==0:
                            break

                        while True:
                            fun=input("请输入您想使用的功能： 1)调整课程 2)查看考试 3)查看签到 4)教学督查 5)显示课程表 q)退出")
                            if fun=='1':
                                self.major.admin[admin_index].adjustCourse()
                            elif fun=='2':
                                self.major.admin[admin_index].staGrade()
                            elif fun=='3':
                                self.major.admin[admin_index].staAttend()
                            elif fun=='4':
                                self.major.admin[admin_index].supervise()
                            elif fun=='5':
                                self.major.admin[admin_index].showSchedule()
                            
                            elif fun=='q' or fun=='Q':
                                break
                            else:
                                print("非法输入，请重新输入！")
                    
        
        
    def register(self):
        print("请选择注册身份：1)学生  2)教师  3)管理员  q)退出")
        character=input()
        if character=='1':
            while(True):
                print("请输入您的学号：")
                id=int(input())
                flag=1
                for stu in self.major.stu:
                    if stu.stu_id==id:
                        print("该用户已存在！")
                        flag=0
                        break
                if flag:
                    name=input("请输入您的姓名：")
                    
                    pwd=input("请输入您的密码：")
                    stu=student(id,name,pwd,self.major.school_id,self.major.academy_id,self.major.major_id)
                    sql='INSERT INTO `eams`.`student`(`stu_id`, `stu_name`, `school_id`,`academy_id`,`major_id`) values(%d,\'%s\',%d,%d,%d)'%(id,name,self.major.school_id,self.major.academy_id,self.major.major_id)
                    
                    try:
                        cursor.execute(sql)
                    
                        print("注册成功！")
                    except:
                        print("未知错误！")
                    self.major.stu.append(stu)
                    break
        elif character=='2':
            while(True):
                print("请输入您的教工号：")
                id=int(input())
                flag=1
                for tea in self.major.tea:
                    if tea.tea_id==id:
                        print("该用户已存在！")
                        flag=0
                        break
                if flag:
                    name=input("请输入您的姓名：")
                    
                    pwd=input("请输入您的密码：")
                    
                    tea=teacher(id,name,pwd,self.major.school_id,self.major.academy_id,self.major.major_id)
                    sql='INSERT INTO `eams`.`teacher`(`teacher_id`, `teacher_name`, `school_id`,`academy_id`,`major_id`) values(%d,\'%s\',%d,%d,%d)'%(id,name,self.major.school_id,self.major.academy_id,self.major.major_id)
                    
                    try:
                        cursor.execute(sql)
                    
                        print("注册成功！")
                    except:
                        print("未知错误！")
                    self.major.tea.append(tea)
            
                    break
        elif character=='3':
            while(True):
                print("请输入您的工号：")
                id=int(input())
                flag=1
                for adm in self.major.admin:
                    if adm.admin_id==id:
                        print("该用户已存在！")
                        flag=0
                        break
                if flag:
                    name=input("请输入您的姓名：")
                    
                    pwd=input("请输入您的密码：")
                    
                    admi=admin(id,name,pwd,self.major.school_id,self.major.academy_id,self.major.major_id)
                    sql='INSERT INTO `eams`.`admin`(`admin_id`, `admin_name`, `school_id`,`academy_id`,`major_id`) values(%d,\'%s\',%d,%d,%d)'%(id,name,self.major.school_id,self.major.academy_id,self.major.major_id)
                    
                    try:
                        cursor.execute(sql)
                    
                        print("注册成功！")
                    except:
                        print("未知错误！")
                    self.major.admin.append(admi)
            
                    break
        else:
            return 
    

        
        
        
    


In [37]:
ea=eams()
ea.run()